In [ ]:
# import required module

import os
import glob as gb
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.utils import shuffle
from skimage.io import imread, imshow
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input, Add, Dropout,Dense, Activation,  Flatten
from tensorflow.keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix


### Loading the downstream dataset 

In [2]:
#get the training data path of the downstream task
trainpath= ('E:/class_decomposition_XDecompo/AP/')

# get the training classes
keys=os.listdir(trainpath)
values=range(len(keys))

training_classes = dict (zip(keys,values))
print('training_classes = ',training_classes)

training_classes =  {'glioma_0': 0, 'glioma_1': 1, 'glioma_2': 2, 'meningioma_0': 3, 'meningioma_1': 4, 'pituitary tumor_0': 5, 'pituitary tumor_1': 6, 'pituitary tumor_2': 7}


In [ ]:
def create_dataset(img_folder):
    new_size=224   
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
       
            image_path= os.path.join(img_folder, dir1,  file)
            image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
            image=cv2.resize(image, (new_size,new_size))
            image=np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            class_name.append(dir1)
    return img_data_array, class_name

In [ ]:
# extract the image array and class name
X_train, y_test =create_dataset(trainpath)
#check items in X_test
print("items in x_test is:       ",len(X_train) , " items") 
print("items in y_test is:       ",len(y_test) , " items") 

In [5]:
#Loading and resizing training data
new_size=224    
X_train = []
y_train = []
train_label=[]
folder_item_numbers = []

for folder in  os.listdir(trainpath ) : 
    train_label.append(folder)
    files = gb.glob(pathname= str( trainpath  + folder + '/*.png'))
    folder_item_numbers.append(len(files))
    for file in files: 
        orignal_image = cv2.imread(file)
        image = cv2.cvtColor(orignal_image, cv2.COLOR_BGR2RGB)
        resized_image = cv2.resize(image , (new_size,new_size))
        X_train.append(resized_image)  # store loaded image
        y_train.append(training_classes[folder])   # store label image

foldernames=pd.DataFrame({'Folder_name':train_label})
itemnumbers=pd.DataFrame({'Traning Image Numbers':folder_item_numbers})
informations=pd.concat([foldernames,itemnumbers],axis=1)
print(informations)
print('----------------------------------------------------------')        
#check items in X_test
print("items in X_train is:       ",len(X_train) , " items") 
print("items in y_train is:       ",len(y_train) , " items") 

         Folder_name  Traning Image Numbers
0           glioma_0                    156
1           glioma_1                    529
2           glioma_2                    455
3       meningioma_0                    290
4       meningioma_1                    275
5  pituitary tumor_0                    208
6  pituitary tumor_1                    322
7  pituitary tumor_2                    214
----------------------------------------------------------
items in X_train is:        2449  items
items in y_train is:        2449  items


In [6]:
#shuffle data
img_size=224
X_train,y_train = shuffle(X_train, y_train)

# Split the training datdset into two groups
x_train, x_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size= 0.8)  
print('Training Samples =:       ',len(x_train) ,'  and the number of labels is     ', len(y_train))
print('Validation Samples =:       ',len(x_valid) ,'  and the number of labels is     ', len(y_valid))

Training Samples =:        1959   and the number of labels is      1959
Validation Samples =:        490   and the number of labels is      490


In [7]:
#converting all TRAIN data to array
x_train = np.array(x_train) 
y_train = np.array(y_train)

x_valid = np.array(x_valid)
y_valid = np.array(y_valid)

y_train = to_categorical(y_train,len(train_label))
y_valid = to_categorical(y_valid,len(train_label))

print("x_train shape  :" ,x_train.shape)
print("y_train shape :", y_train.shape)
print('-----------------------------------------')
print("x_valid shape  :" ,x_valid.shape)
print("y_valid shape :", y_valid.shape)

x_train shape  : (1959, 224, 224, 3)
y_train shape : (1959, 8)
-----------------------------------------
x_valid shape  : (490, 224, 224, 3)
y_valid shape : (490, 8)


In [8]:
#shuffle data
x_train,y_train = shuffle(x_train,y_train)
x_valid,y_valid = shuffle(x_valid,y_valid)
#normalizing data
x_train=x_train/255.0
x_valid =x_valid/255.0


### fine-tuning the learned parameters from the pretext training model into a new downstream task

In [11]:
# Loading the pretext training model
pretext_training=load_model('E:/pretext_braintumour.hdf5')
# Remove the classification output from the pretext model
XDecompo_model = Model(inputs=TransfereModel.input,   outputs = pretext_training.get_layer('dense').output)
XDecompo_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [12]:
# Add Dense layer
#adding the new classification output layer corresponding to the new downstream task
new_prediction =layers.Dense(len(training_classes), 
                             activation='softmax', name="new_task")(XDecompo_model.output)

# build the XDecompo model and visualize it
XDecompo_model = Model(inputs=XDecompo_model.input, outputs=new_prediction)

XDecompo_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [29]:
# Training XDecompo model based on fine-tuning mode.
# In our work, we updating only the weights of upper layers (i.e., the last four layers),
# Freeze all the layers 
for layer in XDecompo_model.layers:
    layer.trainable = False

# let's visualize layer names and layer indices to see the layers we will learn:
for i, layer in enumerate(XDecompo_model.layers):
    print(i, layer.name)

0 input_1
1 conv1_pad
2 conv1_conv
3 conv1_bn
4 conv1_relu
5 pool1_pad
6 pool1_pool
7 conv2_block1_1_conv
8 conv2_block1_1_bn
9 conv2_block1_1_relu
10 conv2_block1_2_conv
11 conv2_block1_2_bn
12 conv2_block1_2_relu
13 conv2_block1_0_conv
14 conv2_block1_3_conv
15 conv2_block1_0_bn
16 conv2_block1_3_bn
17 conv2_block1_add
18 conv2_block1_out
19 conv2_block2_1_conv
20 conv2_block2_1_bn
21 conv2_block2_1_relu
22 conv2_block2_2_conv
23 conv2_block2_2_bn
24 conv2_block2_2_relu
25 conv2_block2_3_conv
26 conv2_block2_3_bn
27 conv2_block2_add
28 conv2_block2_out
29 conv2_block3_1_conv
30 conv2_block3_1_bn
31 conv2_block3_1_relu
32 conv2_block3_2_conv
33 conv2_block3_2_bn
34 conv2_block3_2_relu
35 conv2_block3_3_conv
36 conv2_block3_3_bn
37 conv2_block3_add
38 conv2_block3_out
39 conv3_block1_1_conv
40 conv3_block1_1_bn
41 conv3_block1_1_relu
42 conv3_block1_2_conv
43 conv3_block1_2_bn
44 conv3_block1_2_relu
45 conv3_block1_0_conv
46 conv3_block1_3_conv
47 conv3_block1_0_bn
48 conv3_block1_3_bn

In [30]:
train_layers=[176,171,168,165]

for i in train_layers:
    print('Layer name is:      ',XDecompo_model.layers[i].name)
    print('the input:   ',XDecompo_model.layers[i].input)
    print('the output:  ',XDecompo_model.layers[i].output)
    print('....................................................................')
    

Layer name is:       conv5_block3_1_conv
the input:    KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 2048), dtype=tf.float32, name=None), name='conv5_block2_out/Relu:0', description="created by layer 'conv5_block2_out'")
the output:   KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 512), dtype=tf.float32, name=None), name='conv5_block3_1_conv/BiasAdd:0', description="created by layer 'conv5_block3_1_conv'")
....................................................................


In [31]:
def  training_opt(filename):
    
    file_dir='E:/results/'
    filepath = os.path.join(file_dir,filename+'.hdf5')
    
    #define checkpoint
    checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_accuracy',
                             save_best_only=True, 
                             mode='auto',
                             verbose=1)   

    #early stopping
    earlystop = EarlyStopping( monitor="val_accuracy", 
                        patience=10,  
                        mode="auto")

    model_callbacks = [checkpoint, earlystop]

    return model_callbacks


### Compile and Run

In [ ]:
def lr_function(epoch):
    initial_lrate = 0.001
    drop = 0.90
    epochs_drop = 15.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate
  
callbacks = [checkpoint , LearningRateScheduler(lambda epoch: \
                                              lr_function(epoch),    
                                              verbose=True) ]


for i in train_layers:
    Layername=XDecompo_model.layers[i].name
    print('fine_tuning layer name: -----------------------------    ', Layername)
    
    for layer in XDecompo_model.layers[i:-1]:
        layer.trainable = True
    
    XDecompo_model = Model(inputs=XDecompo_model.input, outputs=new_prediction)
    XDecompo_model.compile( optimizer= 'sgd', loss="mse", metrics=["accuracy"])

    history=XDecompo_model.fit(x_train,y_train, validation_data= (x_valid,y_valid),
                               epochs=50,
                               batch_size=50,
                               callbacks= training_opt(Layername), verbose=1)


In [ ]:
#showing results and model accuracy 
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']
#epochs=30
epochs_range = range(epochs)

plt.figure(figsize=(12, 6))
plt.grid()

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Loading the Test dataset

In [19]:
# get the path of the data set
testpath = ('E:/downstream_data/test/')

keys=os.listdir(testpath)
values=range(len(keys))

Test_classes = dict (zip(keys,values))
print('Test_classes = ',Test_classes)

Test_classes =  {'glioma': 0, 'meningioma': 1, 'pituitary tumor': 2}


In [20]:
#Loading and resizing of test data
new_size=224    
x_test = []
y_test = []
test_labels=[]
folder_item_numbers = []

for folder in  os.listdir(testpath ) : 
    test_labels.append(folder)
    files = gb.glob(pathname= str( testpath  + folder + '/*.png'))
    folder_item_numbers.append(len(files))
    for file in files: 
        orignal_image = cv2.imread(file)
        image = cv2.cvtColor(orignal_image, cv2.COLOR_BGR2RGB)
        resized_image = cv2.resize(image , (new_size,new_size))
        x_test.append(resized_image)
        y_test.append(Test_classes[folder])
        
print('--------------------------------------------------')        
foldernames=pd.DataFrame({'Label_name':test_labels})
itemnumbers=pd.DataFrame({'Number of samples':folder_item_numbers})
informations=pd.concat([foldernames,itemnumbers],axis=1)
print(informations)
print('--------------------------------------------------')        
#check items in X_test
print("items in x_test is:       ",len(x_test) , " items") 
print("items in y_test is:       ",len(y_test) , " items") 

--------------------------------------------------
        Label_name  Number of samples
0           glioma                286
1       meningioma                143
2  pituitary tumor                186
--------------------------------------------------
items in x_test is:        615  items
items in y_test is:        615  items


In [21]:
#converting all Test data to array
x_test = np.array(x_test)
y_test = np.array(y_test)
y_test = to_categorical(y_test,len(test_labels))

print('-----------------------------------------')
print("x_test shape  :" ,x_test.shape)
print("y_test shape :", y_test.shape)
#shuffle data
x_test,y_test = shuffle(x_test,y_test)
#normalizing data
x_test =x_test/255.0

-----------------------------------------
x_test shape  : (615, 224, 224, 3)
y_test shape : (615, 3)


### model evaluation

In [23]:
# load the best parameters of XDecompo_model
XDecompo_model=load_model('E:/results/conv5_block3_1_conv.hdf5') 
 
y_test_pred = XDecompo_model.predict(x_test)
y_prediction = np.argmax(y_test_pred, axis=1)

y_true = np.argmax(y_test, axis=1)

y_true.shape, y_prediction.shape

((615,), (615,))

In [24]:
y_prediction

array([6, 1, 2, 2, 6, 2, 2, 2, 1, 6, 2, 6, 1, 2, 2, 1, 1, 1, 6, 2, 2, 6,
       1, 1, 6, 1, 2, 2, 6, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 6, 2, 2, 1, 6,
       2, 1, 6, 6, 1, 6, 1, 2, 1, 6, 6, 1, 2, 2, 6, 2, 1, 6, 6, 1, 6, 6,
       6, 6, 1, 6, 6, 1, 2, 1, 2, 6, 2, 6, 2, 1, 2, 6, 6, 1, 6, 6, 2, 2,
       2, 1, 6, 2, 1, 6, 2, 2, 2, 6, 2, 6, 6, 1, 2, 6, 2, 6, 2, 1, 6, 2,
       1, 6, 2, 1, 2, 1, 6, 6, 2, 1, 1, 6, 6, 6, 2, 6, 6, 2, 1, 2, 1, 2,
       2, 1, 2, 1, 6, 6, 2, 2, 2, 2, 1, 2, 1, 1, 1, 6, 2, 2, 1, 6, 1, 6,
       1, 2, 1, 2, 6, 2, 6, 2, 6, 6, 6, 1, 1, 6, 6, 2, 2, 1, 2, 1, 2, 6,
       1, 2, 6, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 6, 1, 6, 6, 1, 1, 1, 1,
       1, 2, 2, 6, 1, 6, 1, 1, 2, 1, 1, 2, 2, 2, 6, 6, 1, 1, 2, 2, 6, 1,
       6, 2, 2, 6, 1, 2, 1, 1, 2, 1, 1, 6, 6, 6, 1, 1, 6, 2, 1, 2, 1, 6,
       6, 1, 6, 1, 2, 6, 1, 1, 6, 6, 6, 6, 6, 6, 6, 1, 2, 2, 1, 2, 6, 1,
       6, 1, 6, 2, 2, 2, 6, 1, 2, 2, 2, 2, 1, 1, 2, 6, 2, 6, 1, 2, 2, 2,
       2, 2, 2, 1, 6, 6, 1, 1, 1, 1, 1, 2, 6, 1, 1,

### Class composition


In [26]:
correct_prediction=np.copy(y_prediction)
for idx, i in enumerate(correct_prediction):
    if i==0 or i==1 or i==2 or i==3:
        correct_prediction[idx]=0
    elif   i==4 or i==5:
         correct_prediction[idx]=1
    elif   i==6 or i==7 or i==8:
         correct_prediction[idx]=2
            
correct_prediction

array([2, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2,
       0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2,
       0, 0, 2, 2, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 2, 2, 0, 2, 2,
       2, 2, 0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0,
       0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0, 2, 0,
       0, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2,
       0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0,
       2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 0, 0, 0, 2,
       2, 0, 2, 0, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 0,
       2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0,

### Model evaluation

In [ ]:

#get confusion matrix
cm = confusion_matrix(y_true, correct_prediction)
print(cm)

#plot
fig, ax = plot_confusion_matrix(conf_mat=cm,  figsize=(6, 6),
                                colorbar=False,
                                show_absolute=True,
                                show_normed=False,
                                class_names=Test_classes,cmap="Blues")
plt.show()


#get classification report
print(classification_report(y_true, correct_prediction, target_names=Test_classes, digits=4))
#
#
#

# compute sensitivity and specificity for multi classification 
res = []
for l in list(range(len(org_classes))):
    prec,recall,_,_ = precision_recall_fscore_support(np.array(y_true)==l,
                                       np.array(correct_prediction)==l,
                                              pos_label=True,average=None)
    res.append([l,recall[0],recall[1]])  

    df= pd.DataFrame(res,columns = ['class','sensitivity','specificity']) 
print(df)

#overall sensitivity and specificity
df[1:].mean(axis = 0)

### Gradient-Weighted Class Activation Maps


In [ ]:
def VizGradCAM(model, image, interpolant=0.5, plot_results=True):

    """
    To explain how XDecompo model made its decision,
    we will use Grad-CAM to help visualize the region’s of the input that has
    contributed towards making predictions by the model.
    VizGradCAM - Displays GradCAM based on Keras / TensorFlow models
    using the gradients from the last convolutional layer. This function
    should work with all Keras Application listed here:
    https://keras.io/api/applications/
    Parameters:
    model (keras.model): Compiled Model with Weights Loaded
    image: Image to Perform Inference On
    plot_results (boolean): True - Function Plots using PLT
                            False - Returns Heatmap Array
    Returns:
    Heatmap Array
    """
    #sanity check
    assert (interpolant > 0 and interpolant < 1), "Heatmap Interpolation Must Be Between 0 - 1"
    #STEP 1: Preprocesss image and make prediction using our model
    #input image
    original_img = np.asarray(image, dtype = np.float32)
    #expamd dimension and get batch size
    img = np.expand_dims(original_img, axis=0)
    #predict
    prediction = model.predict(img)
    #prediction index
    prediction_idx = np.argmax(prediction)
    #STEP 2: Create new model
    #specify last convolutional layer
    last_conv_layer = next(x for x in model.layers[::-1] if isinstance(x, K.layers.Conv2D))
    target_layer = model.get_layer(last_conv_layer.name)
    #compute gradient of top predicted class
    with tf.GradientTape() as tape:
        #create a model with original model inputs and the last conv_layer as the output
        gradient_model = Model([model.inputs], [target_layer.output, model.output])
        #pass the image through the base model and get the feature map  
        conv2d_out, prediction = gradient_model(img)
        #prediction loss
        loss = prediction[:, prediction_idx]
    #gradient() computes the gradient using operations recorded in context of this tape
    gradients = tape.gradient(loss, conv2d_out)
    #obtain the output from shape [1 x H x W x CHANNEL] -> [H x W x CHANNEL]
    output = conv2d_out[0]
    #obtain depthwise mean
    weights = tf.reduce_mean(gradients[0], axis=(0, 1))
    #create a 7x7 map for aggregation
    activation_map = np.zeros(output.shape[0:2], dtype=np.float32)
    #multiply weight for every layer
    for idx, weight in enumerate(weights):
        activation_map += weight * output[:, :, idx]
    #resize to image size
    activation_map = cv2.resize(activation_map.numpy(), 
                                (original_img.shape[1], 
                                 original_img.shape[0]))
    #ensure no negative number
    activation_map = np.maximum(activation_map, 0)
    #convert class activation map to 0 - 255
    activation_map = (activation_map - activation_map.min()) / (activation_map.max() - activation_map.min())
    #rescale and convert the type to int
    activation_map = np.uint8(255 * activation_map)
    #convert to heatmap
    heatmap = cv2.applyColorMap(activation_map, cv2.COLORMAP_JET)
    #superimpose heatmap onto image
    original_img = np.uint8((original_img - original_img.min()) / (original_img.max() - original_img.min()) * 255)
    cvt_heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    cvt_heatmap = img_to_array(cvt_heatmap)
    #enlarge plot
    plt.rcParams["figure.dpi"] = 100

    if plot_results == True:
        plt.imshow(np.uint8(original_img * interpolant + cvt_heatmap * (1 - interpolant)))
    else:
        return cvt_heatmap

In [ ]:
#load image
test_img = cv2.imread("E:/downstream_data/test/glioma_tumor/1.jpg")

#apply function
VizGradCAM(XDecompo_model, img_to_array(test_img), plot_results=True)